The dataset for this colab is available: [Training and validation set](https://drive.google.com/file/d/1Ji3oeXia4H4dd4bWpxkgTRfECS6Avn1d/view?usp=sharing) and [Test dataset](https://drive.google.com/file/d/1FWyWSnHNb28bit3mj2xrW5KpcvPKglGo/view?usp=sharing).

The purpose of this study was to forecast the auction sale price of a specific piece of heavy equipment (a bulldozer) based on its usage, equipment type, and configuration. The information comes from auction result postings and covers use and equipment settings.
The RMSLE (root mean squared log error) between the actual and forecast auction prices is the competition's evaluation metric.

The data page contains links to sample submission files. Files submitted should be formatted as follows:

A newly created` header  `SalesID,SalePrice`;
has two columns:

`SalesID`: The validation set's SalesID in sorted order;
`SalePrice`: Your estimated sale price.

# [Root Mean Squared Log Error (RMSLE)](https://www.kaggle.com/code/carlolepelaars/understanding-the-metric-rmsle)

The formula for RMSLE is as follows:

$RMSLE=\sqrt{\frac{1}{n}\sum_{i=1}^{n}(log(p_i +1)-log(a_i +1))^2}$

where:

$n$  is the total number of observations in the (public/private) data set,

$p_i$  is the prediction of target, and

$a_i$  is the actual target for  $i$ .

$log(x)$  is the natural logarithm of  $x  ( log_e(x)$ .

#Prepare tools

##Random Forest Regression

Every decision tree has a significant variance, but when we mix all of them in parallel, the final variance is low since each decision tree is perfectly trained on that specific sample data, and so the outcome is dependent on numerous decision trees rather than one. In the case of a classification problem, the majority voting classifier is used to determine the final output. The final output of a regression problem is the mean of all the outputs. Aggregation is the name given to this section.

Random Forest is an ensemble technique that can handle both regression and classification tasks by combining many decision trees and a technique known as Bootstrap and Aggregation, or bagging. The core idea is to use numerous decision trees to determine the final output rather than depending on individual decision trees.

In [ ]:
# import exploratory data analysis and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# models from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# model evaluations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#Inspect data

#Q1 Import the training and validation dataset `TrainAndValid.csv` with `low_memory=False`and priint first 5 rows of the dataset. What is the `auctionerrID` for `SalesID=1139246`?



In [1]:
import pandas as pd
df = pd.read_csv('TrainAndValid.csv', low_memory=False)

In [2]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [ ]:
df.query('SalesID == 1139246')['auctioneerID']

,auctioneerID
0,3


#Q2 How many non-null values are there in the state column?



In [6]:
df_test = pd.read_csv('Test.csv', low_memory=False)

In [4]:
df['state'].isna().sum()

0

In [10]:
df['state'].notna().sum()

412698

In [18]:
df.query('state != "Unspecified"')['state'].value_counts().sum()

409897

In [7]:
# prompt: How many non-null values are there in the state column?

df_test['state'].notna().count()


12457

In [19]:
df['state'].shape

(412698,)

In [ ]:
df['state'].value_counts()

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412698 entries, 405675 to 22602
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              392562 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  147504 non-null  float64
 8   UsageBand                 73670 non-null   object 
 9   saledate                  412698 non-null  object 
 10  fiModelDesc               412698 non-null  object 
 11  fiBaseModel               412698 non-null  object 
 12  fiSecondaryDesc           271971 non-null  object 
 13  fiModelSeries             58667 non-null   ob

In [21]:
# prompt: Can you sort the dataframe in ascending order of saledate with inplace=True?

df.sort_values(by=["saledate"], inplace=True, ascending=True)


In [22]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
405675,4449186,46173.2,2318824,26964,173,99.0,1996,0.0,NaN,1/1/2012 0:00,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,NaN,NaN
268962,1806686,8000.0,1077085,9549,132,2.0,1998,NaN,NaN,1/10/2003 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221519,1691274,20500.0,1233251,22933,132,2.0,1994,NaN,NaN,1/10/2003 0:00,...,20 inch,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
283617,1895558,9000.0,1458959,6054,132,2.0,1994,NaN,NaN,1/10/2003 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
122970,1467430,17500.0,1339914,4200,132,2.0,1988,NaN,NaN,1/10/2003 0:00,...,20 inch,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [23]:
df.query('SalesID == 1646770 ')['saledate']

,saledate
205615,1/17/1989 0:00


In [70]:
df_copy = df.copy()

In [25]:
df.query('MachineID == 1194089 ')['YearMade']

,YearMade
274835,1980


In [71]:
df_copy['saledate'] = pd.to_datetime(df_copy['saledate'])

In [72]:
# prompt: Separate the saledate in saleYear,saleMonth,saleDay,saleDayOfWeek and saleDayOfYear and saledate column

df_copy["saleYear"] = df_copy.saledate.dt.year
df_copy["saleMonth"] = df_copy.saledate.dt.month
df_copy["saleDay"] = df_copy.saledate.dt.day
df_copy["saleDayOfWeek"] = df_copy.saledate.dt.dayofweek
df_copy["saleDayOfYear"] = df_copy.saledate.dt.dayofyear

df_copy.drop("saledate", axis=1, inplace=True)


In [68]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412698 entries, 405675 to 22602
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              392562 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  147504 non-null  float64
 8   UsageBand                 73670 non-null   object 
 9   fiModelDesc               412698 non-null  object 
 10  fiBaseModel               412698 non-null  object 
 11  fiSecondaryDesc           271971 non-null  object 
 12  fiModelSeries             58667 non-null   object 
 13  fiModelDescriptor         74816 non-null   ob

In [30]:
# prompt: plot distribution line chart over auctioneerID column

df_copy["auctioneerID"].isna().sum()


20136

In [69]:
# prompt: Convert the state and Usageband to numbers. What is the datatype?
# [Hint: use pd.Categorical(df_tmp["feature_name"]).codes]

# Convert string to categories
df_tmp = df_copy
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()
df_tmp["state"] = pd.Categorical(df_tmp["state"]).codes
df_tmp["UsageBand"] = pd.Categorical(df_tmp["UsageBand"]).codes
print(df_tmp["state"].dtype)
print(df_tmp["UsageBand"].dtype)


int8
int8


In [32]:
df_tmp.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfWeek,saleDayOfYear
405675,4449186,46173.2,2318824,26964,173,99.0,1996,0.0,-1,D65E12,...,None or Unspecified,None or Unspecified,None or Unspecified,NaN,NaN,2012,1,1,6,1
268962,1806686,8000.0,1077085,9549,132,2.0,1998,NaN,-1,751,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10
221519,1691274,20500.0,1233251,22933,132,2.0,1994,NaN,-1,EX120-3,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10
283617,1895558,9000.0,1458959,6054,132,2.0,1994,NaN,-1,B502B,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10
122970,1467430,17500.0,1339914,4200,132,2.0,1988,NaN,-1,E120B,...,NaN,NaN,NaN,NaN,NaN,2003,1,10,4,10


In [42]:
pd.Categorical(df_tmp[label][:5]).codes

array([0, 1, 1, 1, 1], dtype=int8)

In [43]:
df_tmp[label][:5]

,saleDayOfYear
405675,1
268962,10
221519,10
283617,10
122970,10


In [48]:
label = 'Differential_Type'
df_tmp[label + '_is_missing'] = df_tmp[label].isnull()

In [83]:
df_tmp[label].dtype

dtype('int32')

In [73]:
for col in df_tmp.columns:
    if df_tmp[col]
      df_tmp[col + '_is_missing'] = df_tmp[col].isnull().astype(int)

df_tmp.info()


<class 'pandas.core.frame.DataFrame'>
Index: 412698 entries, 405675 to 22602
Columns: 114 entries, SalesID to saleDayOfYear_is_missing
dtypes: category(5), float64(3), int32(5), int64(62), int8(2), object(37)
memory usage: 336.0+ MB


In [74]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412698 entries, 405675 to 22602
Columns: 114 entries, SalesID to saleDayOfYear_is_missing
dtypes: category(5), float64(3), int32(5), int64(62), int8(2), object(37)
memory usage: 336.0+ MB


In [78]:
df_tmp.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Backhoe_Mounting_is_missing,Blade_Type_is_missing,Travel_Controls_is_missing,Differential_Type_is_missing,Steering_Controls_is_missing,saleYear_is_missing,saleMonth_is_missing,saleDay_is_missing,saleDayOfWeek_is_missing,saleDayOfYear_is_missing
405675,4449186,46173.2,2318824,26964,173,99.0,1996,0.0,-1,D65E12,...,0,0,0,1,1,0,0,0,0,0
268962,1806686,8000.0,1077085,9549,132,2.0,1998,NaN,-1,751,...,1,1,1,1,1,0,0,0,0,0
221519,1691274,20500.0,1233251,22933,132,2.0,1994,NaN,-1,EX120-3,...,1,1,1,1,1,0,0,0,0,0
283617,1895558,9000.0,1458959,6054,132,2.0,1994,NaN,-1,B502B,...,1,1,1,1,1,0,0,0,0,0
122970,1467430,17500.0,1339914,4200,132,2.0,1988,NaN,-1,E120B,...,1,1,1,1,1,0,0,0,0,0


In [77]:
df_tmp.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc',
       ...
       'Backhoe_Mounting_is_missing', 'Blade_Type_is_missing',
       'Travel_Controls_is_missing', 'Differential_Type_is_missing',
       'Steering_Controls_is_missing', 'saleYear_is_missing',
       'saleMonth_is_missing', 'saleDay_is_missing',
       'saleDayOfWeek_is_missing', 'saleDayOfYear_is_missing'],
      dtype='object', length=114)

In [49]:
df_tmp.query('SalesID == 1646770 ')['Differential_Type_is_missing']


,Differential_Type_is_missing
205615,True


In [60]:
# prompt: Split data into training and validation sets. Data from 2011 will be used for validation. What is the length of the training dataset?

df_val = df_tmp[df_tmp.saleYear == 2011]
df_train = df_tmp[df_tmp.saleYear != 2011]


In [56]:
df_train.shape

(377501, 58)

In [61]:
# prompt: Differentiate the training dataset into two different dataframes X_train and y_train
#  where y_train is the SalePrice feature and X_train has the remaining features. Differentiate the validation dataframe into X_val and y_val in same manner.
# Which of the following options represent the shapes of X_train and y_val?

X_train, y_train = df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_valid, y_valid = df_val.drop("SalePrice", axis=1), df_val.SalePrice

X_train.shape, y_valid.shape


((377501, 98), (35197,))